In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=' + input_.loc[a, 'basePart'] + '&func=PART&vid='
                
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                vehicle = '\n'.join([guide['buyersGuideDesc'].strip() for guide in json_data['pp']['buyersGuides']])

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'rowId']),
                                         'Part_Number': json_data['pd']['basePart'].strip(),
                                         'Category': '' if len(json_data['pp']['categories'])==0 else json_data['pp']['categories'][0]['categoryName_en'].strip(),
                                         'Part_Type_Short': json_data['pd']['partDesc_en'].strip(),
                                         'Part_Type_Long': json_data['pd']['partLongDesc_en'].strip(),
                                         'Brand': json_data['pd']['brand'].strip(),
                                         'POP': json_data['pd']['pop'].strip(),
                                         'Per_Car': json_data['pd']['qtyEach'],
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': '',
                                         'JOIN_PNB': json_data['pd']['webBase'].strip() + ';' + json_data['pd']['brand'].strip()}])

                for specification in json_data['pp']['partSpecs']:
                    df_temp.loc[0, specification['attributeName_en'].lower().strip()] = specification['siteValue_en']

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'rowId': int(input_.loc[a, 'rowId']),
                                     'basePart': input_.loc[a, 'basePart']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'basePart'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：709

[ok] - 90010
[尝试次数：1] - [剩余数量：689] - [当前时间：22:05:30]

[ok] - 90016
[尝试次数：1] - [剩余数量：688] - [当前时间：22:05:30]

[ok] - 90005
[尝试次数：1] - [剩余数量：687] - [当前时间：22:05:30]

[ok] - 90033
[尝试次数：1] - [剩余数量：686] - [当前时间：22:05:31]

[ok] - 90012
[尝试次数：1] - [剩余数量：685] - [当前时间：22:05:31]

[ok] - 44136
[尝试次数：1] - [剩余数量：684] - [当前时间：22:05:31]

[ok] - 90003
[尝试次数：1] - [剩余数量：683] - [当前时间：22:05:31]

[ok] - 90001
[尝试次数：1] - [剩余数量：682] - [当前时间：22:05:31]

[ok] - 90011
[尝试次数：1] - [剩余数量：681] - [当前时间：22:05:31]

[ok] - 90015
[尝试次数：1] - [剩余数量：680] - [当前时间：22:05:31]

[ok] - 90036
[尝试次数：1] - [剩余数量：679] - [当前时间：22:05:31]

[ok] - 90037
[尝试次数：1] - [剩余数量：678] - [当前时间：22:05:31]

[ok] - 90042
[尝试次数：1] - [剩余数量：677] - [当前时间：22:05:31]

[ok] - 90030
[尝试次数：1] - [剩余数量：676] - [当前时间：22:05:32]

[ok] - 90040
[尝试次数：1] - [剩余数量：675] - [当前时间：22:05:32]

[ok] - 90002
[尝试次数：1] - [剩余数量：674] - [当前时间：22:05:32]

[ok] - 90004
[尝试次数：1] - [剩余数量：673] - [当前时间：22:05:32]

[ok] - 90017
[尝试次数：1] - [剩余数量：672] - [当前时间：22:05:32]

[ok] - 90049
[尝试次数：

[ok] - 90073
[尝试次数：1] - [剩余数量：535] - [当前时间：22:06:21]

[ok] - 91801
[尝试次数：1] - [剩余数量：534] - [当前时间：22:06:21]

[ok] - 91500
[尝试次数：1] - [剩余数量：533] - [当前时间：22:06:21]

[ok] - 90041
[尝试次数：2] - [剩余数量：532] - [当前时间：22:06:22]

[ok] - 91800
[尝试次数：1] - [剩余数量：531] - [当前时间：22:06:22]

[ok] - 91794
[尝试次数：1] - [剩余数量：530] - [当前时间：22:06:22]

[ok] - 91819
[尝试次数：1] - [剩余数量：529] - [当前时间：22:06:22]

[ok] - 92006
[尝试次数：1] - [剩余数量：528] - [当前时间：22:06:23]

[ok] - 92002
[尝试次数：1] - [剩余数量：527] - [当前时间：22:06:23]

[ok] - 91818
[尝试次数：1] - [剩余数量：526] - [当前时间：22:06:24]

[ok] - 90069
[尝试次数：2] - [剩余数量：525] - [当前时间：22:06:25]

[ok] - 92010
[尝试次数：1] - [剩余数量：524] - [当前时间：22:06:25]

[ok] - 92007
[尝试次数：1] - [剩余数量：523] - [当前时间：22:06:26]

[ok] - 91790
[尝试次数：1] - [剩余数量：522] - [当前时间：22:06:27]

[ok] - 91583
[尝试次数：2] - [剩余数量：521] - [当前时间：22:06:27]

[ok] - 92015
[尝试次数：1] - [剩余数量：520] - [当前时间：22:06:27]

[ok] - 91769
[尝试次数：2] - [剩余数量：519] - [当前时间：22:06:28]

[ok] - 90014
[尝试次数：2] - [剩余数量：518] - [当前时间：22:06:29]

[ok] - 91630
[尝试次数：1] - [剩余数

[ok] - 92086
[尝试次数：3] - [剩余数量：383] - [当前时间：22:07:08]

[ok] - 92189
[尝试次数：3] - [剩余数量：382] - [当前时间：22:07:09]

[ok] - 92196
[尝试次数：3] - [剩余数量：381] - [当前时间：22:07:10]

[ok] - 92198
[尝试次数：5] - [剩余数量：380] - [当前时间：22:07:10]

[ok] - 92200
[尝试次数：1] - [剩余数量：379] - [当前时间：22:07:11]

[ok] - 92192
[尝试次数：9] - [剩余数量：378] - [当前时间：22:07:12]

[ok] - 92201
[尝试次数：3] - [剩余数量：377] - [当前时间：22:07:13]

[ok] - 92202
[尝试次数：4] - [剩余数量：376] - [当前时间：22:07:13]

[ok] - 92203
[尝试次数：6] - [剩余数量：375] - [当前时间：22:07:13]

[ok] - 92184
[尝试次数：1] - [剩余数量：374] - [当前时间：22:07:13]

[ok] - 92204
[尝试次数：4] - [剩余数量：373] - [当前时间：22:07:14]

[ok] - 92207
[尝试次数：1] - [剩余数量：372] - [当前时间：22:07:15]

[ok] - 92208
[尝试次数：5] - [剩余数量：371] - [当前时间：22:07:16]

[ok] - 92205
[尝试次数：1] - [剩余数量：370] - [当前时间：22:07:16]

[ok] - 92213
[尝试次数：1] - [剩余数量：369] - [当前时间：22:07:17]

[ok] - 92211
[尝试次数：5] - [剩余数量：368] - [当前时间：22:07:17]

[ok] - 92209
[尝试次数：4] - [剩余数量：367] - [当前时间：22:07:17]

[ok] - 92214
[尝试次数：1] - [剩余数量：366] - [当前时间：22:07:18]

[ok] - 92137
[尝试次数：7] - [剩余数

[ok] - 92394
[尝试次数：1] - [剩余数量：231] - [当前时间：22:08:36]

[ok] - 92395
[尝试次数：1] - [剩余数量：230] - [当前时间：22:08:37]

[ok] - 92393
[尝试次数：1] - [剩余数量：229] - [当前时间：22:08:40]

[ok] - 92385
[尝试次数：2] - [剩余数量：228] - [当前时间：22:08:40]

[ok] - 92396
[尝试次数：1] - [剩余数量：227] - [当前时间：22:08:41]

[ok] - 92397
[尝试次数：1] - [剩余数量：226] - [当前时间：22:08:41]

[ok] - 92398
[尝试次数：1] - [剩余数量：225] - [当前时间：22:08:42]

[ok] - 92250
[尝试次数：2] - [剩余数量：224] - [当前时间：22:08:42]

[ok] - 92399
[尝试次数：1] - [剩余数量：223] - [当前时间：22:08:42]

[ok] - 92405
[尝试次数：1] - [剩余数量：222] - [当前时间：22:08:43]

[ok] - 92404
[尝试次数：1] - [剩余数量：221] - [当前时间：22:08:43]

[ok] - 92406
[尝试次数：1] - [剩余数量：220] - [当前时间：22:08:44]

[ok] - 92407
[尝试次数：1] - [剩余数量：219] - [当前时间：22:08:44]

[ok] - 92408
[尝试次数：1] - [剩余数量：218] - [当前时间：22:08:45]

[ok] - 92409
[尝试次数：1] - [剩余数量：217] - [当前时间：22:08:45]

[ok] - 92403
[尝试次数：1] - [剩余数量：216] - [当前时间：22:08:45]

[ok] - 92387
[尝试次数：1] - [剩余数量：215] - [当前时间：22:08:46]

[ok] - 92367
[尝试次数：1] - [剩余数量：214] - [当前时间：22:08:46]

[ok] - 92411
[尝试次数：1] - [剩余数

[ok] - 98530
[尝试次数：1] - [剩余数量：76] - [当前时间：22:09:50]

[ok] - 98534
[尝试次数：1] - [剩余数量：75] - [当前时间：22:09:51]

[ok] - 98533
[尝试次数：1] - [剩余数量：74] - [当前时间：22:09:51]

[ok] - 98539
[尝试次数：1] - [剩余数量：73] - [当前时间：22:09:53]

[ok] - 98535
[尝试次数：1] - [剩余数量：72] - [当前时间：22:09:53]

[ok] - 98544
[尝试次数：1] - [剩余数量：71] - [当前时间：22:09:55]

[ok] - 98537
[尝试次数：1] - [剩余数量：70] - [当前时间：22:09:55]

[ok] - 96138
[尝试次数：1] - [剩余数量：69] - [当前时间：22:09:56]

[ok] - 98552A
[尝试次数：1] - [剩余数量：68] - [当前时间：22:09:57]

[ok] - 98553
[尝试次数：1] - [剩余数量：67] - [当前时间：22:09:57]

[ok] - 98551A
[尝试次数：1] - [剩余数量：66] - [当前时间：22:09:57]

[ok] - 92402
[尝试次数：2] - [剩余数量：65] - [当前时间：22:09:58]

[ok] - 92380
[尝试次数：3] - [剩余数量：64] - [当前时间：22:09:58]

[ok] - 98554A
[尝试次数：1] - [剩余数量：63] - [当前时间：22:09:59]

[ok] - 98559A
[尝试次数：1] - [剩余数量：62] - [当前时间：22:09:59]

[ok] - 98572A
[尝试次数：1] - [剩余数量：61] - [当前时间：22:09:59]

[ok] - 96127
[尝试次数：1] - [剩余数量：60] - [当前时间：22:10:00]

[ok] - 98555A
[尝试次数：1] - [剩余数量：59] - [当前时间：22:10:00]

[ok] - 92390
[尝试次数：3] - [剩余数量：58] - [当前时